## Recommendation System

In [1]:
import firebase_admin
from firebase_admin import db,credentials
import main as recommender
import pandas as pd
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://fir-demo-29d5b-default-rtdb.firebaseio.com'})

### Get data from Firebase Realtime DB

In [2]:
ref = db.reference('/')
data = ref.child('songs').order_by_child('id').get()
songs = []
for key in data:
    songs.append(data[key])
stat = ref.child('stats').order_by_child('songId').get()
stats = []
for key in stat:
    stats.append(stat[key])

In [3]:
songs_df = pd.DataFrame(songs).rename(columns={'id' : 'songId'}).dropna()
stats_df = pd.DataFrame(stats)
song_stat_df = pd.merge(stats_df, songs_df.drop_duplicates(['songId']), on='songId', how='left')
song_stat_df.drop(columns=['320kbps','album_id' ,'duration','has_lyrics' , 'image','perma_url', 'url', 'release_date', 'genre'],inplace=True)
# to convert string into list of strings 
def spliting(text):
  text=str(text).split()
  return text

song_stat_df['title'] = song_stat_df['title'].apply(spliting)
song_stat_df['artist'] = song_stat_df['artist'].apply(spliting)
song_stat_df['album'] = song_stat_df['album'].apply(spliting)
song_stat_df['subtitle'] = song_stat_df['subtitle'].apply(spliting)
song_stat_df['year'] = song_stat_df['year'].apply(spliting)
song_stat_df['all_tags'] = song_stat_df['title'] + song_stat_df['artist'] + song_stat_df['album'] + song_stat_df['subtitle']+ song_stat_df['year']
song_stat_df.drop(columns=['title','artist' ,'album','language' , 'subtitle','year'],inplace=True)
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
  list_i = []
  for i in input_str:
    s = ''
    for c in i:
      if c in s1:
        s += s0[s1.index(c)]
      else:
        s += c
    list_i.append(s)
  
  return list_i

song_stat_df['all_tags']=song_stat_df['all_tags'].apply(remove_accents)
def convert_lower(text):
  l=[]
  for item in text:
    l.append(item.lower())
  return l

song_stat_df['all_tags']=song_stat_df['all_tags'].apply(convert_lower)
song_stat_df

,listenCount,songId,userId,all_tags
0,15,-7sISWuTdj0,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,"[phuc, du, -, yeu, anh, di, me, anh, ban, banh..."
1,7,-7sISWuTdj0,d9155580-f7c7-11ed-ab2f-f146d31a9014,"[phuc, du, -, yeu, anh, di, me, anh, ban, banh..."
2,5,-7sISWuTdj0,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,"[phuc, du, -, yeu, anh, di, me, anh, ban, banh..."
3,18,-E2u6exhzPE,0947e130-f7cf-11ed-92de-f7e7740e5f43,"[making, my, way, son, tung, m-tp, son, tung, ..."
4,21,-GQg25oP0S4,0c2983d0-f16d-11ed-9c88-514c38a48ebe,"[seventeen, (세븐틴), '손오공', official, mv, hybe, ..."
...,...,...,...,...
1400,12,zlTIextYnyQ,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,"[kai, 카이, 'rover', mv, smtown, smtown, smtown,..."
1401,9,znlFu_lemsU,282e4620-f5f4-11ed-b8ac-a71a1638bcad,"[mary, j., blige, -, family, affair, (official..."
1402,21,znvky0Uq8qc,d9155580-f7c7-11ed-ab2f-f146d31a9014,"[until, i, found, you, stephen, sanchez, steph..."
1403,30,znvky0Uq8qc,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,"[until, i, found, you, stephen, sanchez, steph..."


In [4]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steming(text):
  l=[]
  for i in text:
    l.append(ps.stem(i))
  return l

song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(steming)
song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(lambda x: " ".join(x))
song_stat_df

,listenCount,songId,userId,all_tags
0,15,-7sISWuTdj0,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,phuc du - yeu anh di me anh ban banh mi (mv of...
1,7,-7sISWuTdj0,d9155580-f7c7-11ed-ab2f-f146d31a9014,phuc du - yeu anh di me anh ban banh mi (mv of...
2,5,-7sISWuTdj0,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,phuc du - yeu anh di me anh ban banh mi (mv of...
3,18,-E2u6exhzPE,0947e130-f7cf-11ed-92de-f7e7740e5f43,make my way son tung m-tp son tung m-tp son tu...
4,21,-GQg25oP0S4,0c2983d0-f16d-11ed-9c88-514c38a48ebe,seventeen (세븐틴) '손오공' offici mv hybe label hyb...
...,...,...,...,...
1400,12,zlTIextYnyQ,f6110910-f7bb-11ed-b8bd-ffe1409aca9d,kai 카이 'rover' mv smtown smtown smtown 2023
1401,9,znlFu_lemsU,282e4620-f5f4-11ed-b8ac-a71a1638bcad,mari j. blige - famili affair (offici music vi...
1402,21,znvky0Uq8qc,d9155580-f7c7-11ed-ab2f-f146d31a9014,until i found you stephen sanchez stephen sanc...
1403,30,znvky0Uq8qc,08ed0940-f7ca-11ed-b7d7-b75e9c825c2a,until i found you stephen sanchez stephen sanc...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
all_tags = song_stat_df['all_tags']
data = tfidf.fit_transform(all_tags).toarray()
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(data)
similarity

array([[1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.07009129],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.07009129],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.07009129],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.07009129, 0.07009129, 0.07009129, ..., 0.        , 0.        ,
        1.        ]])

In [15]:
def movie_recommend(movie):
    index = song_stat_df[song_stat_df['songId'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    list_id = []
    for i in distances:
        if len(list_id) > 20:
            break
        id = song_stat_df.iloc[i[0]].songId
        if id not in list_id:
            list_id.append(id)
    columns = ['320kbps','album','album_id','artist','duration','genre','has_lyrics','songId','image','language','perma_url','release_date','subtitle','title','url','year']
    list_result = pd.DataFrame(columns= columns)
    for id in list_id:
        result = songs_df.query('songId == @id')
        list_result = pd.concat([list_result, result], ignore_index=True).drop_duplicates(['songId'])
    return list_result
    
movie_recommend("6t-MjBazs3o").head()

,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,215,YouTube,false,6t-MjBazs3o,https://img.youtube.com/vi/6t-MjBazs3o/maxresd...,YouTube,https://www.youtube.com/watch?v=6t-MjBazs3o,2020-07-05 00:00:00.000,Sơn Tùng M-TP Official,SƠN TÙNG M-TP | CÓ CHẮC YÊU LÀ ĐÂY | OFFICIAL ...,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2020
1,false,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,278,YouTube,false,FN7ALfpGxiI,https://img.youtube.com/vi/FN7ALfpGxiI/maxresd...,YouTube,https://www.youtube.com/watch?v=FN7ALfpGxiI,2017-02-13 00:00:00.000,Sơn Tùng M-TP Official,NƠI NÀY CÓ ANH | OFFICIAL MUSIC VIDEO | SƠN TÙ...,https://rr3---sn-8qj-nbo67.googlevideo.com/vid...,2017
2,false,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,192,YouTube,false,488ceQWoGGw,https://img.youtube.com/vi/488ceQWoGGw/maxresd...,YouTube,https://www.youtube.com/watch?v=488ceQWoGGw,2016-07-25 00:00:00.000,Sơn Tùng M-TP Official,Nắng Ấm Xa Dần - Sơn Tùng M-TP,https://rr8---sn-8qj-nboey.googlevideo.com/vid...,2016
3,false,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,258,YouTube,false,niPkap1ozUA,https://img.youtube.com/vi/niPkap1ozUA/maxresd...,YouTube,https://www.youtube.com/watch?v=niPkap1ozUA,2023-05-04 00:00:00.000,Sơn Tùng M-TP Official,SON TUNG M-TP | MAKING MY WAY | OFFICIAL VISUA...,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2023
4,false,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,243,YouTube,false,qGRU3sRbaYw,https://img.youtube.com/vi/qGRU3sRbaYw/maxresd...,YouTube,https://www.youtube.com/watch?v=qGRU3sRbaYw,2016-08-02 00:00:00.000,Sơn Tùng M-TP Official,Chúng Ta Không Thuộc Về Nhau | Official Music ...,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2016


In [3]:
rec_sys = recommender.recommender_system()
rec_sys.data_preprocessing(songs=songs, stats= stats)

In [4]:
re = rec_sys.get_popularity()
out = pd.DataFrame(re)
out.head()

,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Có Chắc Yêu Là Đây,UC3muIvzjhubNpJ4Pn_0kCQw,Sơn Tùng M-TP,215,YouTube,false,sTgsNeblHu8,https://lh3.googleusercontent.com/r6FYNr4_qCsa...,YouTube,https://www.youtube.com/watch?v=sTgsNeblHu8,2020-08-02 00:00:00.000,Song • Sơn Tùng M-TP • Có Chắc Yêu Là Đây • 3:36,Có Chắc Yêu Là Đây,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2020
1,true,Shape of You,3320300,Ed Sheeran,233,English,false,icJam_5l,https://c.saavncdn.com/126/Shape-of-You-Englis...,English,https://www.jiosaavn.com/song/shape-of-you/GQs...,2017-01-06,Ed Sheeran - Shape of You,Shape of You,https://aac.saavncdn.com/126/da7cde34b008294e1...,2017
2,true,÷,10436096,Ed Sheeran,263,English,false,6o8JoQ8b,https://c.saavncdn.com/286/WMG_190295851286-En...,English,https://www.jiosaavn.com/song/perfect/RgdTexthD1E,2017-03-03,Ed Sheeran - ÷,Perfect,https://aac.saavncdn.com/286/71bb6cc3391ddf619...,2017
3,true,Flowers,41619705,Miley Cyrus,200,English,false,c-6-mB8N,https://c.saavncdn.com/047/Flowers-English-202...,English,https://www.jiosaavn.com/song/flowers/E0VdHBlyD30,2023-01-13,Miley Cyrus - Flowers,Flowers,https://aac.saavncdn.com/047/985bd26eb8496d9a7...,2023
4,true,STAY,28263397,"Justin Bieber, The Kid Laroi",140,English,false,kd8JSDbB,https://c.saavncdn.com/895/Stay-English-2021-2...,English,https://www.jiosaavn.com/song/stay/GwxTeyd0VXE,2021-07-09,"Justin Bieber, The Kid Laroi - STAY",STAY,https://aac.saavncdn.com/895/9a3f48fb98e9dd0b2...,2021


In [5]:
pd.DataFrame(rec_sys.get_user_similarity('0c2983d0-f16d-11ed-9c88-514c38a48ebe'))

No. of unique songs for the user: 296
no. of unique songs in the training set: 795
Non zero values in cooccurence_matrix :99820


,320kbps,album,album_id,artist,duration,genre,has_lyrics,songId,image,language,perma_url,release_date,subtitle,title,url,year
0,true,Dandelions (slowed + reverb),29160455,"Ruth B., sped up + slowed",268,English,false,POhlVMwJ,https://c.saavncdn.com/839/Dandelions-slowed-r...,English,https://www.jiosaavn.com/song/dandelions-slowe...,2021-08-27,"Ruth B., sped up + slowed - Dandelions (slowed...",Dandelions (slowed + reverb),https://aac.saavncdn.com/839/2d3d560559fc503ae...,2021
1,true,Flowers,41619705,Miley Cyrus,200,English,false,c-6-mB8N,https://c.saavncdn.com/047/Flowers-English-202...,English,https://www.jiosaavn.com/song/flowers/E0VdHBlyD30,2023-01-13,Miley Cyrus - Flowers,Flowers,https://aac.saavncdn.com/047/985bd26eb8496d9a7...,2023
2,false,土岐麻子,UC9Lb-o-O-H35yTqFLXrlCnQ,土岐麻子,297,YouTube,false,ti62cJ_3oEM,https://img.youtube.com/vi/ti62cJ_3oEM/maxresd...,YouTube,https://www.youtube.com/watch?v=ti62cJ_3oEM,2020-08-31 00:00:00.000,土岐麻子,土岐麻子 / 「HOME」Music Video,https://rr1---sn-hvcp4vox8o-nbos.googlevideo.c...,2020
3,false,Amplifier,UCoLH6RkTGYdqW0qAfd5yO8Q,Amplifier,493,YouTube,false,46SGSGti2xQ,https://img.youtube.com/vi/46SGSGti2xQ/maxresd...,YouTube,https://www.youtube.com/watch?v=46SGSGti2xQ,2018-06-11 00:00:00.000,Amplifier - Topic,Matmos,https://rr1---sn-8qj-nbo66.googlevideo.com/vid...,2018
4,false,HIEUTHUHAI,UCh9j7NC6jVhHREbmqd9C7YA,HIEUTHUHAI,228,YouTube,false,r96eaATJWW8,https://img.youtube.com/vi/r96eaATJWW8/maxresd...,YouTube,https://www.youtube.com/watch?v=r96eaATJWW8,2023-04-19 00:00:00.000,HIEUTHUHAI - Topic,KHÔNG THỂ SAY,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2023
5,false,buitruonglinh,UCd23_I7g0mes9U5nL1XaPfw,buitruonglinh,213,YouTube,false,T3GHC43K5ZI,https://img.youtube.com/vi/T3GHC43K5ZI/maxresd...,YouTube,https://www.youtube.com/watch?v=T3GHC43K5ZI,2022-07-08 00:00:00.000,buitruonglinh - Topic,Dù Cho Mai Về Sau (Acoustic Version),https://rr8---sn-8qj-nboey.googlevideo.com/vid...,2022
6,false,Mono Official,UCi_RaMQ4Hhdml1Cff4SCpQA,Mono Official,265,YouTube,false,CHw1b_1LVBA,https://img.youtube.com/vi/CHw1b_1LVBA/maxresd...,YouTube,https://www.youtube.com/watch?v=CHw1b_1LVBA,2022-08-18 00:00:00.000,Mono Official,MONO - Waiting For You (Album 22 - Track No.10),https://rr5---sn-8qj-nbo67.googlevideo.com/vid...,2022
7,false,RodWave,UCenjunBhBhvKjfDAESnoppw,RodWave,208,YouTube,false,6dW4d0IcQ3k,https://img.youtube.com/vi/6dW4d0IcQ3k/maxresd...,YouTube,https://www.youtube.com/watch?v=6dW4d0IcQ3k,2023-03-30 00:00:00.000,RodWave,Rod Wave - Fight The Feeling (Official Video),https://rr5---sn-42u-nbozs.googlevideo.com/vid...,2023
8,false,Adam Christopher,UCw8h_nufRgtbdZX7enuXBFQ,Adam Christopher,184,YouTube,false,5HKeq3x6YPI,https://img.youtube.com/vi/5HKeq3x6YPI/maxresd...,YouTube,https://www.youtube.com/watch?v=5HKeq3x6YPI,2019-03-24 00:00:00.000,Adam Christopher - Topic,Iris (Acoustic),https://rr5---sn-42u-nbozs.googlevideo.com/vid...,2019
9,false,DJSnakeVEVO,UCGkSXL5saM8y3VvJD_PdMFQ,DJSnakeVEVO,205,YouTube,false,euCqAq6BRa4,https://img.youtube.com/vi/euCqAq6BRa4/maxresd...,YouTube,https://www.youtube.com/watch?v=euCqAq6BRa4,2016-11-29 00:00:00.000,DJSnakeVEVO,DJ Snake - Let Me Love You ft. Justin Bieber,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2016


## API

In [ ]:
import service as sv
self_sv = sv.firebase_service()
self_sv.get_data_and_preprocessing()

In [ ]:
self_sv.random_update_stats()